## 准备数据

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [3]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [26]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.flatten = tf.keras.layers.Flatten()
        self.W1 = tf.Variable(tf.random.normal([784, 512]))
        self.b1 = tf.Variable(tf.zeros([512]))
        self.W2 = tf.Variable(tf.random.normal([512, 10]))
        self.b2 = tf.Variable(tf.zeros([10]))
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = self.flatten(x)
        x = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(x, self.W2) + self.b2
        return logits
    

        
model = myModel()

optimizer = optimizers.Adam()

In [11]:
weights = model.dense2.get_weights()[1].shape
print(weights)

(1,)


## 计算 loss

In [22]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [27]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 218.20299 ; accuracy 0.15223333
epoch 1 : loss 189.6261 ; accuracy 0.16585
epoch 2 : loss 169.08162 ; accuracy 0.17706667
epoch 3 : loss 151.71777 ; accuracy 0.18715
epoch 4 : loss 136.81003 ; accuracy 0.19616666
epoch 5 : loss 124.431786 ; accuracy 0.20563333
epoch 6 : loss 114.80313 ; accuracy 0.21363333
epoch 7 : loss 107.45887 ; accuracy 0.22305
epoch 8 : loss 101.70895 ; accuracy 0.23283334
epoch 9 : loss 97.020035 ; accuracy 0.2412
epoch 10 : loss 93.08256 ; accuracy 0.24886666
epoch 11 : loss 89.70433 ; accuracy 0.25725
epoch 12 : loss 86.725136 ; accuracy 0.26651666
epoch 13 : loss 84.04399 ; accuracy 0.27518332
epoch 14 : loss 81.573814 ; accuracy 0.28386667
epoch 15 : loss 79.26124 ; accuracy 0.29203334
epoch 16 : loss 77.07722 ; accuracy 0.30071667
epoch 17 : loss 75.00818 ; accuracy 0.30898333
epoch 18 : loss 73.04372 ; accuracy 0.3174
epoch 19 : loss 71.175186 ; accuracy 0.32568333
epoch 20 : loss 69.39824 ; accuracy 0.33463332
epoch 21 : loss 67.71221 ; acc